# Figure Six: PFSS Model

- Panel (a): Full Carrington EUV map with PFSS model overlaid
    - footpoints (grey, pink (FSW), green (SSW), purple (SASW))
    - trajectory (red (positive polarity), blue (negative polarity))
    - HCS (white)
    - footpoint radial magnetic field ($B_{r, \, 0}$)
- Bottom Panels: zoom in of top panel onto four periods of interest along with footpoint brightness
    - Panel (b): HCS crossing
    - Panel (c): FSW region
    - Panel (d): SSW & SASW regions

## Imports

In [1]:
import os

import datetime
import numpy as np
import pandas as pd
import astropy.units as u
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

import astrospice, sunpy
import sunpy.coordinates as scoords
from astropy.coordinates import SkyCoord

import tools.sigma as sigma
import tools.utilities as utils
import tools.pfss_funcs as pfss_funcs
from tools.psp_funcs import ballistically_project
for sc in ['psp','solar orbiter'] : kernels = astrospice.registry.get_kernels(sc,'predict') 

# COLORS
c = ['#ae017e','#085A99',  '#c98000'] # darker colors
lightc = ['#FCA4C4',  '#8FD3F4', '#FFCC70'] # lighter colors
fcol = 'mistyrose'
sacol='lavender'
sacol = 'violet'
scol = 'lightgreen'
hcol = 'lightblue'
aa = 0.6
lw=2
clon = '#ae017e'
clat = '#085A99'
rcol = 'dimgrey'
cmaps = ['RdPu', 'cool', 'Wistia', 'spring']

# REGIONS
loc_hcs = [113, 116]
ssw = [166, 175]
sasw = [175, 185]
fsw = [70, 85]

# DIRECTORIES
IMG_DIR = './figures'
DF_DIR = './results'
PlotDir = '/Users/tamarervin/mplstyle/'

STORAGE_DIR = '/Volumes/TAMARUSB/pub_data/e11_ch'
RES_DIR = os.path.join(STORAGE_DIR, 'results')

# PLOT STYLING
plot_style = os.path.join(PlotDir, 'figure_series.mplstyle')
plt.rcParams['mathtext.fontset'] = 'custom'
plt.rcParams['mathtext.cal'] = 'Helvetica Neue LT Pro'
plt.rcParams.update({'font.size': 18})
plt.style.use(plot_style)

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

08-Jan-24 10:56:05: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),



Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

## Data

In [2]:
# ### READ IN DATA ###
parker, parkerdownt, parkerdownl, pss, orbiter, orbiterdownl, orbiterdownt, oss, his_orbiter, his_orbiterdownt, his_orbiterdownl = utils.read_data(RES_DIR, sigma_time=20)

### DEFINE REGIONS ###
lon_footpoints = parker.sslon
hhcs = np.logical_and(lon_footpoints >= loc_hcs[0], lon_footpoints <=loc_hcs[1])
slow = np.logical_and(lon_footpoints >=sasw[0], lon_footpoints <=sasw[1])
fast = np.logical_and(lon_footpoints >=fsw[0], lon_footpoints <=fsw[1])

parker

08-Jan-24 10:56:15: /Users/tamarervin/publications/e11_ch/tools/utilities.py:180: FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.

>>> df.resample(freq="3s", loffset="8H")

becomes:

>>> from pandas.tseries.frequencies import to_offset
>>> df = df.resample(freq="3s").mean()
>>> df.index = df.index.to_timestamp() + to_offset("8H")

  pss = parker.resample(sigma_bin, closed='left', label='left', loffset=sigma_bin / 2).mean()

08-Jan-24 10:56:15: /Users/tamarervin/publications/e11_ch/tools/utilities.py:180: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pss = parker.resample(sigma_bin, closed='left', label='left', loffset=sigma_bin / 2).mean()

08-Jan-24 10:56:26: /Users/tamarervin/publications/e11_ch/tools/utilities.py:189: FutureWarning: 'loffset' in .resample() and in Group

,Time,vr,vt,vn,Np,Tp,angle_vb,vra,vta,vna,...,lon,lat,rAU,sslon,sslat,ssrAU,NpR2,NeR2,BrR2,bins
Time,,,,,,,,,,,,,,,,,,,,,
2022-02-23 00:00:03.926892,2022-02-23 00:00:03.926892,370.97455,-68.608260,12.858900,145.62561,86.790770,1.884407,465.81497,-129.249470,-36.253708,...,47.708430,2.388760,0.156872,57.771146,2.388760,0.009301,3.583668,4.558838,-2.350418,57
2022-02-23 00:00:07.422132,2022-02-23 00:00:07.422132,382.27350,-74.660080,14.060577,161.48608,89.291084,1.826346,462.15216,-120.491400,3.749751,...,47.708403,2.388731,0.156870,57.607543,2.388731,0.009301,3.973891,4.558742,-2.378433,57
2022-02-23 00:00:10.917373,2022-02-23 00:00:10.917373,364.42395,-74.207940,19.968895,177.64952,89.371605,1.865545,403.23470,-56.667350,-62.046680,...,47.708377,2.388702,0.156869,57.972668,2.388702,0.009301,4.371553,4.558645,-2.306081,57
2022-02-23 00:00:14.412674,2022-02-23 00:00:14.412674,372.37628,-75.410360,28.382940,169.79265,79.205550,1.941305,530.37024,-74.644510,-9.971888,...,47.708350,2.388673,0.156867,57.796664,2.388673,0.009301,4.178125,4.558549,-2.346586,57
2022-02-23 00:00:17.907915,2022-02-23 00:00:17.907915,374.23737,-78.762375,24.308477,158.62505,88.632700,1.909643,506.34766,-103.938210,-22.601944,...,47.708324,2.388644,0.156865,57.792816,2.388644,0.009301,3.903239,4.558452,-2.314443,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-28 23:59:43.194114,2022-02-28 23:59:43.194114,228.82224,-8.253631,-10.827599,276.13450,18.814621,1.398025,273.78880,-24.720728,-10.724208,...,187.868751,0.477085,0.184722,207.395602,0.477085,0.009301,9.422366,7.775737,2.893766,NaN
2022-02-28 23:59:46.689415,2022-02-28 23:59:46.689415,229.35385,-9.343748,-10.732706,279.30658,18.966917,1.340095,267.28770,-34.630830,-1.861496,...,187.868571,0.477111,0.184724,207.362644,0.477111,0.009301,9.530766,7.775868,2.906969,NaN
2022-02-28 23:59:50.184655,2022-02-28 23:59:50.184655,228.71349,-10.778193,-9.042365,282.03693,18.999338,1.329281,260.90866,-23.886229,-5.436355,...,187.868391,0.477137,0.184726,207.404395,0.477137,0.009301,9.624097,7.776000,2.918893,NaN


## Run PFSS Model

In [4]:
# download/read in magnetogram
rss=2.0
filename = 'adapt40311_03k012_202202240000_i00005600n1.fts.gz'
filepath = f"{STORAGE_DIR}/{filename}"
adapt_magnetogram = pfss_funcs.adapt2pfsspy(filepath, return_magnetogram=True)
gong_map = sunpy.map.Map(adapt_magnetogram.data/1e5, adapt_magnetogram.meta)

# run PFSS model
pfss_model = pfss_funcs.adapt2pfsspy(filepath,rss)

# trace PFSS lines
flines = pfss_funcs.pfss2flines(pfss_model)

08-Jan-24 14:15:21: Missing metadata for solar radius: assuming the standard radius of the photosphere.
08-Jan-24 14:15:21: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/sunpy/map/mapbase.py:633: SunpyMetadataWarning: Missing metadata for observer: assuming Earth-based observer.
For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following metadata is missing: crlt_obs,crln_obs,dsun_obs

  obs_coord = self.observer_coordinate

08-Jan-24 14:15:22: Missing metadata for solar radius: assuming the standard radius of the photosphere.
08-Jan-24 14:15:22: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/sunpy/map/mapbase.py:633: SunpyMetadataWarning: Missing metadata for observer: assuming Earth-based observer.
For frame 'heliographic_stonyhurst' the following metadata is missing: dsun_obs,hglt_obs,hgln_obs
For frame 'heliographic_carrington' the following met

INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]
INFO: Missing metadata for solar radius: assuming the standard radius of the photosphere. [sunpy.map.mapbase]


08-Jan-24 14:15:25: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/pfsspy/tracing.py:180: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to 1000) and try again.
  warnings.warn(



In [5]:
# get PSP at source surface
# get datetimes from fields dataframe
pdatetimes = parker.Time

# get inertial PSP coordinates 
kernels = astrospice.registry.get_kernels('psp', 'predict')
psp_coords_inertial = astrospice.generate_coords('SOLAR PROBE PLUS', pdatetimes)

# Transform to Heliographic Carrington, i.e. the frame that co-rotates with the Sun.
psp_coords_carr = psp_coords_inertial.transform_to(
sunpy.coordinates.HeliographicCarrington(observer="self"))

# get psp longitude and latitude at source surface
psp_at_source_surface = ballistically_project(psp_coords_carr,vr_arr=np.array(parker.vr)*u.km/u.s, r_inner=2.0*u.Rsun)

Files Downloaded:   0%|          | 0/1 [00:00<?, ?file/s]

In [6]:
# get Br at the source surface from the pfss model
pfss_br = pfss_model.source_surface_br

# get HCS
hcs = pfss_model.source_surface_pils[0]

# get trajectory 
polarity = np.sign(parker.BrR2)
pos = np.where(polarity == 1)
neg = np.where(polarity == -1)

# get field lines
flines_psp = pfss_funcs.pfss2flines(pfss_model, skycoord_in=psp_at_source_surface)

# high res field lines
flines_highres = pfss_funcs.pfss2flines(pfss_model,nth=181,nph=361)

# get field line topology defined by polarity
topologies = flines_highres.polarities.reshape([181,361])

08-Jan-24 14:21:27: /Users/tamarervin/miniconda3/envs/analysis/lib/python3.11/site-packages/pfsspy/tracing.py:180: UserWarning: At least one field line ran out of steps during tracing.
You should probably increase max_steps (currently set to 1000) and try again.
  warnings.warn(



## Plotting Function

In [7]:
def plot_pfss(smap, hcs, source_surface, flines, datetimes, ax, nf=8, dim=[0, 360, -90, 90], tm=7, dd=True, yl=True, fp=True, full=True):

    # plot on axes
    plt.sca(ax)
    
    # color dictionary
    color_dict = {-1:"blue", 0:"black", 1:"red"}
    
    # plot euv map
    if type(smap) == sunpy.map.mapbase.GenericMap:
        lons = np.linspace(0, 360, 721)
        lats = np.linspace(-90, 90, 361)
        lognorm = mpl.colors.LogNorm(vmin=np.nanpercentile(smap.data.flatten(),10
                                                           ), 
                                vmax=np.nanpercentile(smap.data.flatten(),99.9))
        ax.pcolormesh(lons, lats, smap.data, cmap='sdoaia193', norm=lognorm, zorder=-1)
    else:
        lons = np.linspace(0, 360, 361)
        lats = np.linspace(-90, 90, 181)
        ax.pcolormesh(lons,lats,topologies,cmap="coolwarm", zorder=-1)
        
    # plot HCS
    ax.plot(hcs.lon, hcs.lat, color='white', label='HCS', zorder=0)

    if full:
        # plot field lines
        for f in flines_psp[::nf] :
            fcoords = f.coords
            fcoords.representation_type="spherical"
            ax.plot(fcoords.lon,
                    fcoords.lat,
                    # fcoords.z.to("R_sun"),
                    color = color_dict.get(f.polarity), 
                    linewidth=0.5, alpha=0.5, zorder=1
                )

        # plot trajectory
        polarity = np.sign(parker.BrR2)
        pos = np.where(polarity == 1)
        neg = np.where(polarity == -1)
        ax.scatter(source_surface.lon[neg], source_surface.lat[neg], color='navy', label='Negative Polarity', zorder=2, s=3)
        ax.scatter(source_surface.lon[pos], source_surface.lat[pos], color='darkred',label='Positive Polarity', zorder=3, s=3)

    # add regions of interest
    lon_footpoints = flines.open_field_lines.source_surface_feet.lon.value[::nf]
    lat_footpoints = flines.open_field_lines.source_surface_feet.lat[::nf]
    fast = np.logical_and(lon_footpoints >= fsw[0], lon_footpoints <=fsw[1])
    slow = np.logical_and(lon_footpoints >= ssw[0], lon_footpoints <=ssw[1])
    salf = np.logical_and(lon_footpoints >= sasw[0], lon_footpoints <=sasw[1])

    # plot footpoints
    lon_footpoints = flines.open_field_lines.solar_feet.lon.value[::nf]
    lat_footpoints = flines.open_field_lines.solar_feet.lat[::nf]
    ax.scatter(lon_footpoints, lat_footpoints, s=6, color='grey', marker='D')
    ax.scatter(lon_footpoints[fast], lat_footpoints[fast], s=6, color=fcol, marker='D')
    ax.scatter(lon_footpoints[slow], lat_footpoints[slow], s=6, color=scol, marker='D')
    ax.scatter(lon_footpoints[salf], lat_footpoints[salf], s=6, color=sacol, marker='D')

    # plot dates
    if dd:
        dates = [i.date() for i in datetimes]
        dates_str = [d.strftime('%m-%d-%Y') for d in dates]
        psp_inds = [np.where(np.array(dates_str) == d)[0][0] for d in np.unique(dates_str)[1:]]
        labels = np.unique(dates_str)[1:]
        for i, x in enumerate(list(zip(psp_at_source_surface.lon[psp_inds], psp_at_source_surface.lat[psp_inds]))):
            label = labels[i]
            ax.text(x[0].value - 2, x[1].value + 2, label[:-5], ha="center", va="bottom", color='white', rotation=45, size='large', zorder=5)

    # footpoints legend
    if fp:
        fleg = mpatches.Patch(color=fcol, label=r'$\rm Fast \; Wind$')
        saleg = mpatches.Patch(color=sacol, label=r'$\rm Slow \; Alfvenic \; Wind$')
        sleg = mpatches.Patch(color=scol, label=r'$\rm Classical \; Slow \; Wind$')
        footpoints = mpatches.Patch(color='grey', label=r'$\rm Footpoints$')
        leg2 = ax.legend(handles=[footpoints, fleg, sleg, saleg], loc='upper right')
        ax.add_artist(leg2)

    # title and labels
    ax.set_xlim((dim[0], dim[1]))
    ax.set_ylim((dim[2], dim[3]))
    ax.set_xticks(np.linspace(dim[0], dim[1], tm))
    ax.set_yticks(np.linspace(dim[2], dim[3], tm))
    ax.set_xlabel(r"$\rm Carrington \; Longitude \; [deg]$")
    if yl:
        ax.set_ylabel(r"$\rm Carrington \; Latitude \; [deg]$")

    return ax

## Footpoint Brightness

In [ ]:
#### FOOTPOINTS
lats = np.array(flines_psp.open_field_lines.solar_feet.lat)
lons = np.array(flines_psp.open_field_lines.solar_feet.lon.value)

#### FOOTPOINT BRIGHTNESS
synoptic = '/Users/tamarervin/e11_conjunction/data/E11_14days.fits'
smap = sunpy.map.Map(synoptic)

# Convert latitude and longitude arrays to SkyCoord
coords = SkyCoord(lon=lons*u.deg, lat=lats*u.deg, frame=smap.coordinate_frame)

# Convert SkyCoord coordinates to pixel coordinates
pixel_coords = coords.to_pixel(smap.wcs)

# Extract data values from SunPy map at pixel coordinates
data_values = smap.data[pixel_coords[1].astype(int), pixel_coords[0].astype(int)]

#### FIELD STRENGTH
smap = gong_map

# Convert latitude and longitude arrays to SkyCoord
coords = SkyCoord(lon=lons*u.deg, lat=lats*u.deg, frame=smap.coordinate_frame)

# Convert SkyCoord coordinates to pixel coordinates
pixel_coords = coords.to_pixel(smap.wcs)
field_strength = smap.data[pixel_coords[1].astype(int), pixel_coords[0].astype(int)]


In [ ]:
### SETUP FIGURE
import matplotlib as mpl
fig = plt.figure(figsize=(20, 14))
grid = plt.GridSpec(2, 3, height_ratios=[2, 1], width_ratios=[1, 1, 1], hspace=0.2, wspace=0.1)
nf = 16

### CREATE SUBPLOTS
ax1 = fig.add_subplot(grid[0, :])
ax2 = fig.add_subplot(grid[1, 0])
ax3 = fig.add_subplot(grid[1, 1])
ax4 = fig.add_subplot(grid[1, 2])
axs = [ax1, ax2, ax3, ax4]

##### ---------- PANEL (A): PFSS FIGURE  ---------- ######
### READ IN EUV MAP
synoptic = '/Users/tamarervin/e11_conjunction/data/E11_14days.fits'
smap = sunpy.map.Map(synoptic)

### PLOT PFSS MODEL
plot_pfss(smap, hcs, psp_at_source_surface, flines_psp, pdatetimes, ax=ax1, nf=20, dim=[0, 240, -90, 90])

### ADD RECTANGLES
lab = ['(b)', '(c)', '(d)']
for i, rect_x in enumerate([60, 100, 165]):
    rect = mpl.patches.Rectangle((rect_x, -30), 30, 60, linewidth=1, edgecolor='white', facecolor='none')
    ax1.add_patch(rect) 
    rect_corners = [(rect_x, -30),
                    (rect_x + 30, -30),
                    (rect_x + 30, 30),
                    (rect_x, 30)]
    ax1.text(rect_x + 3, 27, lab[i], fontsize=16, fontweight='bold', va='top', ha='left', color='black')

##### ---------- PANEL (B): FAST WIND REGION  ---------- ######
plot_pfss(smap, hcs, psp_at_source_surface, flines_psp, pdatetimes, ax=ax2, nf=20, dim=[60, 90, -45, 45], tm=4, fp=False, dd=False, full=True)
ax2.set_title(r'$\rm Fast \; Wind$')

##### ---------- PANEL (C): HCS REGION  ---------- ######
plot_pfss(smap, hcs, psp_at_source_surface, flines_psp, pdatetimes, ax=ax3, nf=20, dim=[100, 130, -45, 45], tm=4, fp=False, dd=False, yl=False, full=True)
ax3.set_title(r'$\rm HCS$')
ax3.set_yticks([-30, -10, 10, 30])
ax3.set_yticklabels([])

##### ---------- PANEL (D): SLOW WIND REGIONS  ---------- ######
plot_pfss(smap, hcs, psp_at_source_surface, flines_psp, pdatetimes, ax=ax4, nf=20, dim=[165, 195, -45, 45], tm=4, fp=False, dd=False, yl=False, full=True)
ax4.set_title(r'$\rm Slow \; Wind$')
ax4.set_yticks([-30, -10, 10, 30])
ax4.set_yticklabels([])

##### ---------- PANEL (A): FOOTPOINT FIELD STRENGTH  ---------- ######
ss=40
nf=32
d1 = flines_psp.open_field_lines.solar_feet.lon.value[::nf]
d2 = field_strength[::nf]*100

lon_footpoints = flines_psp.open_field_lines.source_surface_feet.lon.value[::nf]
fast = np.logical_and(lon_footpoints >= fsw[0], lon_footpoints <=fsw[1])
salf = np.logical_and(lon_footpoints >= sasw[0], lon_footpoints <=sasw[1])
hhcs = np.logical_and(lon_footpoints >= loc_hcs[0], lon_footpoints <=loc_hcs[1])
slow = np.logical_and(lon_footpoints >=ssw[0], lon_footpoints <=ssw[1])

axf = ax1.twinx()
axf.set_ylim(-0.2, 3) 
axf.set_yticklabels([-0.1, 0, 0.1, 0.2])
axf.set_yticks([-0.1, 0, 0.1, 0.2])
axf.set_ylabel(r'$\rm Photospheric \; B_R \;  [nT]$')
axf.axhspan(-0.1, 0.2, alpha=1, color='silver', zorder=-1)
axf.scatter(d1, d2, c='grey', s=ss, linewidth=0.8, edgecolor='k')
axf.scatter(d1[fast], d2[fast], c=fcol, s=ss, linewidth=0.8, edgecolor='k')
axf.scatter(d1[slow], d2[slow], c=scol, s=ss, linewidth=0.8, edgecolor='k')
axf.scatter(d1[salf], d2[salf], c=sacol, s=ss, linewidth=0.8, edgecolor='k')
axf.scatter(d1[hhcs], d2[hhcs], c=hcol, s=ss, linewidth=0.8, edgecolor='k')

##### ---------- PANELS (C, D, E): RELATIVE FOOTPOINT BRIGHTNESS  ---------- ######
nf=16
d1 = flines_psp.open_field_lines.solar_feet.lon.value[0::nf]
d2 = data_values[0::nf]/np.nanmax(data_values)
lon_footpoints = flines_psp.open_field_lines.source_surface_feet.lon.value[::nf]
fast = np.logical_and(lon_footpoints >= fsw[0], lon_footpoints <=fsw[1])
salf = np.logical_and(lon_footpoints >= sasw[0], lon_footpoints <=sasw[1])
hhcs = np.logical_and(lon_footpoints >= loc_hcs[0], lon_footpoints <=loc_hcs[1])
slow = np.logical_and(lon_footpoints >=ssw[0], lon_footpoints <=ssw[1])

for ax in [ax2, ax3, ax4]:
    axf = ax.twinx()
    ### ADD BACKGROUND COLOR
    axf.axhspan(0, 1.125, alpha=1, color='silver', zorder=-1)
    ### PLOT DATA
    axf.scatter(d1, d2, c='grey', s=ss, linewidth=0.8, edgecolor='k')
    axf.scatter(d1[fast], d2[fast], c=fcol, s=ss, linewidth=0.8, edgecolor='k')
    axf.scatter(d1[slow], d2[slow], c=scol, s=ss, linewidth=0.8, edgecolor='k')
    axf.scatter(d1[salf], d2[salf], c=sacol, s=ss, linewidth=0.8, edgecolor='k')
    axf.scatter(d1[hhcs], d2[hhcs], c=hcol, s=ss, linewidth=0.8, edgecolor='k')
    ### LIMITS AND LABELS
    axf.set_ylim(0, 5)
    axf.set_yticklabels([])
    axf.set_yticks([])
axf.set_ylabel(r'$\rm Relative \; Footpoint \; Brightness$')

### PANEL LABELS
ax2.text(0.02, 0.93, '(a)', transform=ax1.transAxes, fontsize=16, fontweight='bold', va='top', ha='left', color='black')
ax2.text(0.05, 0.93, '(b)', transform=ax2.transAxes, fontsize=16, fontweight='bold', va='top', ha='left', color='black')
ax3.text(0.05, 0.93, '(c)', transform=ax3.transAxes, fontsize=16, fontweight='bold', va='top', ha='left', color='black')
ax4.text(0.05, 0.93, '(d)', transform=ax4.transAxes, fontsize=16, fontweight='bold', va='top', ha='left', color='black')

### SAVE FIGURE
fig.savefig('figures/fig6.png', bbox_inches='tight') #PNG
fig.savefig('eps_figures/fig6.eps', bbox_inches='tight') #EPS
fig.savefig('eps_figures/fig6.pdf') #EPS